In [0]:
!git clone https://github.com/16cs009/alyceproject.git
import os
os.chdir("/content/alyceproject")
!pip install -r requirements.txt
!python -m spacy download en

import neuralcoref
import nltk
!python -m nltk.downloader all


!pip install "python-socketio[client]"

In [0]:
import os
import spacy
import neuralcoref
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag
from nltk.corpus import stopwords
from Datasets import datasets

nlp = spacy.load('en')
coref = neuralcoref.NeuralCoref(nlp.vocab)
nlp.add_pipe(coref, name='neuralcoref')

In [0]:
def phaseI_processStory(story):
	# Returns a list of list with the following as the content of list inside the list :>
	# 1. Dialogue-Replaced sentence from story
	# 2. Type of the Sentence : NARRATION, DIALOGUE, NEWLINE
	# 3. YES or NO string indicating whether NeuralCoref was applied to resolve reference
	# 4. If NeuralCoref was applied then The object It returned
	# 5. If the sentence is of type dialogue then its speaker
	# 6. If the sentence is of type dialogue then the corresponding dialogue
	dialogues = re.findall(r'\"(.+?)\"',story)
	dialogue_count = 0
	for dialogue in dialogues:
		qt_mark = "\"" if(story.find("\""+dialogue+"\"")!=-1) else '\''
		story = story.replace(qt_mark+dialogue+qt_mark,"<-DIALOGUE-#"+str(dialogue_count)+"#->")
		dialogue_count = dialogue_count + 1

	storylist = []
	for paragraph in story.split("\n"):
		for sentence in paragraph.split("."):
			if(len(sentence.strip())!=0):
				tmp_slist = list()
				tmp_slist.append(sentence)
				if(sentence.find("<-DIALOGUE-#")!=-1):
					tmp_slist.append("DIALOGUE")
				else:
					tmp_slist.append("NARRATION")
				tmp_slist.append("NO")
				tmp_slist.append(" ")
				tmp_slist.append(" ")
				tmp_slist.append(" ")
				storylist.append(tmp_slist)
		storylist.append(["\\n","NEWLINE"," "," "," "," "])

	for i in range(len(storylist)):
		if(storylist[i][1]=="DIALOGUE" and i>0):
			words = storylist[i][0].upper().split(" ")
			if("HE" in words or "SHE" in words or "HIM" in words or "HER" in words):
				j=i-1
				while(j>0):
					if(storylist[j][1]!="NEWLINE"):
						break
					j=j-1
				coreference_resolver_result = nlp(".".join([storylist[j][0],storylist[i][0]]))
				storylist[i][0] = coreference_resolver_result._.coref_resolved.split(".")[-1]
				storylist[i][2] = "YES"
				storylist[i][3] = str(coreference_resolver_result._.coref_clusters)

	k = 0
	s1 = {}
	for i in range(len(storylist)):
		if(storylist[i][1]=="DIALOGUE"):
			dialogue_holder = re.findall(r'<-DIALOGUE-#[0-9][0-9]*#->',storylist[i][0])[0]
			dialogue_containing_resolved_sentence = storylist[i][0].replace(dialogue_holder,"").strip()
			storylist[i][5] = dialogues[k]
			stop_words = set(stopwords.words("english"))
			d1 = nltk.word_tokenize(dialogue_containing_resolved_sentence.lower())
			d2 = [j for j in d1 if not j in stop_words]
			for w in nltk.pos_tag(d2):
				if(w[1]=="NN"):
					if(w[0].lower() in s1.keys()):
						s1[w[0].lower()] = s1[w[0].lower()]+1
					else:
						s1[w[0].lower()] = 1
			k = k + 1
	character_list = [k for k in s1.keys() if s1[k]>1]

	k = 0
	for i in range(len(storylist)):
		if(storylist[i][1]=="DIALOGUE"):
			storylist[i][4]="UNKNOWN"
			sentence = " ".join([ w if w not in datasets.wordList else "said" for w in storylist[i][0].split(" ")])
			for subsentence in sentence.split(","):
				if(subsentence.find("said")!=-1):
					for word in subsentence.split(" "):
						if(word.strip().lower() in character_list):
							storylist[i][4]=word.strip()
							break
					break
			k = k + 1
	return storylist

In [0]:
import socketio
import json
import time

sio = socketio.Client()

def workHandler(input_data):
	sio.emit('phase-1-server-to-main-server',{"client": input_data["client"], "procedureNo": input_data["procedureNo"], "status": "STARTED", "output": json.dumps([["None"]])})
	output_data = phaseI_processStory(input_data["input"])
	#sio.sleep(30);
	sio.emit('phase-1-server-to-main-server',{"client": input_data["client"], "procedureNo": input_data["procedureNo"], "status": "COMPLETED", "output": json.dumps(output_data)})

@sio.on('main-server-to-phase-1-server')
def on_phaseOneData(data):
	print("RECEIVED DATA : {}".format(data))
	sio.start_background_task(workHandler, data)

print("Starting Sub server of Phase 1 !...")
sio.connect(input("Enter NGROK Link : "))
sio.emit('server-ready',{"phase": 1})
print("Started Sub server of Phase 1 !...")

try:
	while True:
		time.sleep(1)
except KeyboardInterrupt:
	print("Interrupt Received!...")
	sio.disconnect()

print("Exiting Sub Server!....")